# How to work with gurobi pandas to define decision var with multiple sets (index)

## Root folder and read env variables

In [1]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-2]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  D:\github-mi-repo\Gurobi-ML-tips-modeling


In [2]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## RUN

In [3]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

## 2. Create gurobi model

In [4]:
# create model
m = gp.Model('modelo')

Restricted license - for non-production use only - expires 2025-11-24


### 3. Create decision variables
- Decision variables that are features in ml models
- Decicion variable that is the output in ml models

#### 3.1 define multiple sets

In [5]:
# define set time
list_set_time = ['t0', 't1', 't2', 't3', 't4', 't5', 't6']
index_set_time = pd.Index(list_set_time)
index_set_time

Index(['t0', 't1', 't2', 't3', 't4', 't5', 't6'], dtype='object')

In [6]:
# define set additional - second set defined to this example - of example, think the set represent the kind of product
list_set_product = ['w', 'x', 'y', 'z']
index_set_product = pd.Index(list_set_product)
index_set_product

Index(['w', 'x', 'y', 'z'], dtype='object')

In [7]:
############## multi set region&time - index pandas ##############
index_set_product_time = pd.MultiIndex.from_product((list_set_product, list_set_time), 
                                                    names = ('product', 'time')
                                                   )
index_set_product_time

MultiIndex([('w', 't0'),
            ('w', 't1'),
            ('w', 't2'),
            ('w', 't3'),
            ('w', 't4'),
            ('w', 't5'),
            ('w', 't6'),
            ('x', 't0'),
            ('x', 't1'),
            ('x', 't2'),
            ('x', 't3'),
            ('x', 't4'),
            ('x', 't5'),
            ('x', 't6'),
            ('y', 't0'),
            ('y', 't1'),
            ('y', 't2'),
            ('y', 't3'),
            ('y', 't4'),
            ('y', 't5'),
            ('y', 't6'),
            ('z', 't0'),
            ('z', 't1'),
            ('z', 't2'),
            ('z', 't3'),
            ('z', 't4'),
            ('z', 't5'),
            ('z', 't6')],
           names=['product', 'time'])

#### 3.2 create decision variables MULTIPLE INDEX

In [12]:
# create decision variables
var_Z1_multiple = gppd.add_vars(m, index_set_product_time, name = "decision variable Z1 - MULTI INDEX"
                                     )

var_X2_multiple = gppd.add_vars(m, index_set_product_time, name = "decision variable X2 - MULTI INDEX"
                                     )

var_Y2_multiple = gppd.add_vars(m, index_set_product_time, name = "decision variable Y2 - MULTI INDEX"
                                     )

In [13]:
# "compile"
m.update()

In [14]:
# see decision var created
var_Y2_multiple

product  time
w        t0      <gurobi.Var decision variable Y2 - MULTI INDEX...
         t1      <gurobi.Var decision variable Y2 - MULTI INDEX...
         t2      <gurobi.Var decision variable Y2 - MULTI INDEX...
         t3      <gurobi.Var decision variable Y2 - MULTI INDEX...
         t4      <gurobi.Var decision variable Y2 - MULTI INDEX...
         t5      <gurobi.Var decision variable Y2 - MULTI INDEX...
         t6      <gurobi.Var decision variable Y2 - MULTI INDEX...
x        t0      <gurobi.Var decision variable Y2 - MULTI INDEX...
         t1      <gurobi.Var decision variable Y2 - MULTI INDEX...
         t2      <gurobi.Var decision variable Y2 - MULTI INDEX...
         t3      <gurobi.Var decision variable Y2 - MULTI INDEX...
         t4      <gurobi.Var decision variable Y2 - MULTI INDEX...
         t5      <gurobi.Var decision variable Y2 - MULTI INDEX...
         t6      <gurobi.Var decision variable Y2 - MULTI INDEX...
y        t0      <gurobi.Var decision variable Y

#### 3.3 create decision variables ONE INDEX

In [18]:
# create decision variables
var_Z1_one = gppd.add_vars(m, index_set_product, name = "decision variable Z1 - ONE INDEX"
                                     )

var_X2_one = gppd.add_vars(m, index_set_product, name = "decision variable X2 - ONE INDEX"
                                     )

var_Y2_one = gppd.add_vars(m, index_set_product, name = "decision variable Y2 - ONE INDEX"
                                     )

In [19]:
# "compile"
m.update()

In [20]:
# see decision var created
var_Y2_one

w    <gurobi.Var decision variable Y2 - ONE INDEX[w]>
x    <gurobi.Var decision variable Y2 - ONE INDEX[x]>
y    <gurobi.Var decision variable Y2 - ONE INDEX[y]>
z    <gurobi.Var decision variable Y2 - ONE INDEX[z]>
Name: decision variable Y2 - ONE INDEX, dtype: object

### 5. Define objective optimization
Objetive that no generate infeasibility

In [22]:
var_Y2_one.sum() # sum across time

<gurobi.LinExpr: decision variable Y2 - ONE INDEX[w] + decision variable Y2 - ONE INDEX[x] + decision variable Y2 - ONE INDEX[y] + decision variable Y2 - ONE INDEX[z]>

In [24]:
m.setObjective(var_Y2_one.sum(),
               gp.GRB.MINIMIZE)

#### 6. Optimize and get optimal values

In [25]:
# solve
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19043.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 0 rows, 264 columns and 0 nonzeros
Model fingerprint: 0x30bbd984
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 264 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [26]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
m.Status

2

In [34]:
# get optimal values and save in a dataframe
######## create a dataframe with set used previosly
solution = pd.DataFrame(index = index_set_product)

######################## save optimal values - features of models (only the features) ########################

# model
solution["var_Z1_one"] = var_Z1_one.gppd.X
solution["var_X2_one"] = var_X2_one.gppd.X


######################## save optimal values - targets of models (some targets are features of the model of the next step) ########################
solution["var_Y2_one"] = var_Y2_one.gppd.X  # model


######################## # get value objetive function ########################
opt_objetive_function = m.ObjVal

In [35]:
# show value objetive function
opt_objetive_function

0.0

In [36]:
# show value decision variables
solution

,var_Z1_one,var_X2_one,var_Y2_one
w,0.0,0.0,0.0
x,0.0,0.0,0.0
y,0.0,0.0,0.0
z,0.0,0.0,0.0
